In [4]:
import json
import os
import re
from tqdm import tqdm

In [5]:
# 데이터 경로 (os.path.join 사용, 절대 경로)
BASE_DIR = os.getcwd()
print(BASE_DIR)
# BASE_DIR = os.path.abspath(os.path.dirname(__file__))
TRAINING_DATA_PATH = os.path.join(BASE_DIR, 'data', '채용면접 인터뷰 데이터', 'Training')
VALIDATION_DATA_PATH = os.path.join(BASE_DIR,'data', '채용면접 인터뷰 데이터', 'Validation')
OUTPUT_FILE_PATH = os.path.join(BASE_DIR,'data', 'preprocessed_data.json')

c:\Users\Playdata\github\SKN12-FINAL-5TEAM


In [12]:
# 전처리 파라미터
MIN_SENTENCE_LENGTH = 5  # 이 길이 미만의 문장은 필터링

# 제거할 추임새 및 습관어 목록 (정규식으로 구성)
# 주의: 단어 경계(\b)를 사용하여 부분 일치를 방지 (예: '그'가 '그림'에서 삭제되는 것을 방지)
FILLER_WORDS_PATTERN = re.compile(
    r'\b(음|네|아|어|그|저|뭐랄까|그러니까|사실|그냥|약간|일단|아무튼|솔직히|어떻게 보면|좀|너무|글쎄요|있잖아요|뭐라고 해야 할까|노력하는 편입니다|잘 모르겠지만)\b'
)

# 제거할 불필요한 기호 목록
SYMBOL_PATTERN = re.compile(r'[\"()[\\].,?!...]{2,}') # 2번 이상 반복되는 기호
WHITESPACE_PATTERN = re.compile(r'\s+') # 중복 공백

In [13]:
def remove_fillers(text):
    """추임새 및 습관어를 제거합니다."""
    return FILLER_WORDS_PATTERN.sub('', text).strip()

def remove_symbols(text):
    """불필요한 기호를 제거합니다."""
    return SYMBOL_PATTERN.sub(' ', text)

def normalize_whitespace(text):
    """중복 공백 및 앞뒤 공백을 정리합니다."""
    return WHITESPACE_PATTERN.sub(' ', text).strip()

def preprocess_text(text):
    """정의된 파이프라인에 따라 텍스트를 전처리합니다."""
    if not isinstance(text, str):
        return ""
    
    # 1. 추임새 및 습관어 제거
    text = remove_fillers(text)
    # 2. 불필요한 기호 제거
    text = remove_symbols(text)
    # 3. 공백 정리
    text = normalize_whitespace(text)
    # 4. (선택) 맞춤법 교정
    # text = correct_spelling(text)
    
    return text

In [14]:
def get_all_file_paths(directory_paths):
    """모든 JSON 파일 경로를 재귀적으로 찾습니다."""
    file_paths = []
    for path in directory_paths:
        if not os.path.exists(path):
            print(f"경로가 존재하지 않습니다: {path}")
            continue
        for root, _, files in os.walk(path):
            for file in files:
                if file.endswith('.json'):
                    file_paths.append(os.path.join(root, file))
    return file_paths

def main():
    """전체 전처리 과정을 실행하고 결과를 파일로 저장합니다."""
    print("면접 음성 텍스트 데이터 전처리를 시작합니다.")
    
    # 1. 데이터 로딩
    all_json_files = get_all_file_paths([TRAINING_DATA_PATH, VALIDATION_DATA_PATH])
    if not all_json_files:
        print("오류: 지정된 경로에서 JSON 파일을 찾을 수 없습니다.")
        return

    print(f"총 {len(all_json_files)}개의 파일을 발견했습니다. 전처리를 진행합니다...")
    
    processed_data = []
    
    for file_path in tqdm(all_json_files, desc="전처리 진행 중"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"JSON 디코딩 오류: {file_path}: {e}")
                    continue
                # 2. 원본 텍스트 추출
                raw_question = data.get('dataSet', {}).get('question', {}).get('raw', {}).get('text', "")
                raw_answer = data.get('dataSet', {}).get('answer', {}).get('raw', {}).get('text', "")
                raw_answer_count = data.get('dataSet', {}).get('answer', {}).get('raw', {}).get('wordCount', "")
                raw_answer_summary_count = data.get('dataSet', {}).get('answer', {}).get('summary', {}).get('wordCount', "")
                raw_expression = data.get('dataSet', {}).get('answer', {}).get('intent', [])[0].get('expression', "")
                # 3. 전처리 적용
                clean_question = preprocess_text(raw_question)
                clean_answer = preprocess_text(raw_answer)
                
                # 4. 유효성 검사 (Null, 빈 문자열, 최소 길이)
                if not clean_question or not clean_answer:
                    continue
                if len(clean_question) < MIN_SENTENCE_LENGTH or len(clean_answer) < MIN_SENTENCE_LENGTH:
                    continue
                # 5. 최종 데이터 구조화
                file_id = os.path.basename(file_path).split('.')[0]
                processed_data.append({
                    "id": file_id,
                    "question": clean_question,
                    "answer": clean_answer,
                    "answer_word_count": raw_answer_count,
                    "answer_summary_word_count": raw_answer_summary_count,
                    "expression": raw_expression,
                })
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
        except Exception as e:
            print(f"알 수 없는 오류 발생 {file_path}: {e}")

    print(f"전처리 완료. 총 {len(processed_data)}개의 유효한 데이터가 처리되었습니다.")

    # 6. 전처리된 데이터를 JSON 파일로 저장
    with open(OUTPUT_FILE_PATH, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)
        
    print(f"전처리된 데이터가 '{OUTPUT_FILE_PATH}' 파일로 저장되었습니다.")

    # 7. 전/후 예시 출력
    if processed_data:
        print("\n--- 전/후 처리 예시 ---")
        example_id = processed_data[3]['id']
        # 원본 파일을 다시 읽어와서 비교
        example_raw_path = os.path.join(TRAINING_DATA_PATH, example_id + '.json')
        if not os.path.exists(example_raw_path):
            example_raw_path = os.path.join(VALIDATION_DATA_PATH, example_id + '.json')
        if os.path.exists(example_raw_path):
            try:
                with open(example_raw_path, 'r', encoding='utf-8') as f:
                    raw_example = json.load(f)
                    print(f"원본 답변: {raw_example['dataSet']['answer']['raw']['text']}")
                    print(f"전처리 후 답변: {processed_data[3]['answer']}")
            except Exception as e:
                print(f"예시 파일 처리 중 오류: {e}")
        else:
            print("예시 파일 경로를 찾을 수 없어 전/후 비교를 생략합니다.")

In [15]:
main()

면접 음성 텍스트 데이터 전처리를 시작합니다.
총 6573개의 파일을 발견했습니다. 전처리를 진행합니다...


전처리 진행 중: 100%|██████████| 6573/6573 [00:00<00:00, 7431.85it/s]


전처리 완료. 총 6573개의 유효한 데이터가 처리되었습니다.
전처리된 데이터가 'c:\Users\Playdata\github\SKN12-FINAL-5TEAM\data\preprocessed_data.json' 파일로 저장되었습니다.

--- 전/후 처리 예시 ---
원본 답변: 음 아무래도 네 데이터의 보관과 보호가 제일 중요하기 때문에 음 인터넷에 그런 접근이 원활한지 가 우선 제일 최우선시 되어야 되지 않을까 합니다. 네 그게 어떻게 보면 안전할 수도 있는 게 그런 해킹이라든지 여러 가지 분야에서 안전할 수도 있다 고 보통은 생각들을 하시는데요. 그렇지 않습니다. 그래서 음 이게 잘 접속이 되고 이게 원활하게 들어와야지 어 저희가 많이 구축해 놓은 그런 보안 보안이 철저하게 이루어지기 때문에 그것들이 잘 진행이 되려면 우선 네 원활한 인터넷 접속이 우선시 되어야 된다고 생각을 합니다. 그래야지 저희가 보호하려는 엄청난 양의 데이터들을 저희가 꾸준하게 유지할 수 있어요.
전처리 후 답변: 아무래도 데이터의 보관과 보호가 제일 중요하기 때문에 인터넷에 그런 접근이 원활한지 가 우선 제일 최우선시 되어야 되지 않을까 합니다. 그게 안전할 수도 있는 게 그런 해킹이라든지 여러 가지 분야에서 안전할 수도 있다 고 보통은 생각들을 하시는데요. 그렇지 않습니다. 그래서 이게 잘 접속이 되고 이게 원활하게 들어와야지 저희가 많이 구축해 놓은 그런 보안 보안이 철저하게 이루어지기 때문에 그것들이 잘 진행이 되려면 우선 원활한 인터넷 접속이 우선시 되어야 된다고 생각을 합니다. 그래야지 저희가 보호하려는 엄청난 양의 데이터들을 저희가 꾸준하게 유지할 수 있어요.
